# Landmarks

In [2]:
# DEPENDENCIES ------>
import os
import pickle
import mediapipe as mp
import cv2

In [ ]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)
data_dir = "./data_small"
data = []
labels = []


In [ ]:

for dir_ in os.listdir(data_dir):
    for img_path in os.listdir(os.path.join(data_dir, dir_)):
        data_aux = []
        img = cv2.imread(os.path.join(data_dir, dir_, img_path))
        if img is None:
            print(f"Error reading image: {os.path.join(data_dir, dir_, img_path)}")
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
            data.append(data_aux)
            labels.append(dir_)
file = open("data_small.pickle", "wb")
pickle.dump({"data": data, "labels": labels}, file)
file.close()


# Testing

In [17]:
# DEPENDENCIES ------>
import pickle
import cv2
import mediapipe as mp
import numpy as np
from sklearn.metrics import accuracy_score

In [31]:
model_dict = pickle.load(open("./model.p", "rb"))
model = model_dict["model"]
capture = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9
)


# labels_dict = {0: "Zero", 1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five", 6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}

labels_dict = {
    0: "aA",
    1: "bB",
    2: "cC",
    3: "dD",
    4: "eE",
    5: "fF",
    6: "gG",
    7: "hH",
    8: "iI",
    9: "jJ",
    10: "kK",
    11: "lL",
    12: "mM",
    13: "nN",
    14: "oO",
    15: "pP",
    16: "qQ",
    17: "rR",
    18: "sS",
    19: "tT",
    20: "uU",
    21: "vV",
    22: "wW",
    23: "xX",
    24: "yY",
    25: "zZ",
}

In [17]:
help(model.predict_proba)

Help on method predict_proba in module sklearn.ensemble._forest:

predict_proba(X) method of sklearn.ensemble._forest.RandomForestClassifier instance
    Predict class probabilities for X.
    
    The predicted class probabilities of an input sample are computed as
    the mean predicted class probabilities of the trees in the forest.
    The class probability of a single tree is the fraction of samples of
    the same class in a leaf.
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        The input samples. Internally, its dtype will be converted to
        ``dtype=np.float32``. If a sparse matrix is provided, it will be
        converted into a sparse ``csr_matrix``.
    
    Returns
    -------
    p : ndarray of shape (n_samples, n_classes), or a list of such arrays
        The class probabilities of the input samples. The order of the
        classes corresponds to that in the attribute :term:`classes_`.



In [32]:
while True:
    data_aux = []
    x_ = []
    y_ = []
    ret, frame = capture.read()

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    ret, frame = capture.read()
    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style(),
            )

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)
                x_.append(x)
                y_.append(y)
            x1 = int(min(x_) * W) - 30
            y1 = int(min(y_) * H) - 30
            x2 = int(max(x_) * W) + 30
            y2 = int(max(y_) * H) + 30
            prediction = model.predict([np.asarray(data_aux)])
            # what is its prediction accuracy
            probabilities = model.predict_proba([np.asarray(data_aux)])
            predicted_class_probability = probabilities[0][int(prediction[0])]

            # print the predition accuracy
            print(prediction[0], predicted_class_probability)
            predicted_sign = labels_dict[int(prediction[0])]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
            cv2.putText(
                frame,
                predicted_sign,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.3,
                (0, 0, 255),
                3,
                cv2.LINE_AA,
            )
            data_aux = []
            x_ = []
            y_ = []

    cv2.imshow("Handy", frame)
    cv2.waitKey(1)

capture.release()
cv2.destroyAllWindows()


16 0.01
16 0.01
7 0.0
1 0.33
1 0.34
1 0.34
1 0.38
1 0.4
1 0.36
1 0.36
1 0.35
1 0.36
1 0.36
1 0.36
1 0.37
1 0.39
7 0.01
1 0.21
7 0.01
7 0.01
7 0.01
7 0.01
7 0.01
7 0.01
7 0.01
7 0.01
7 0.0
7 0.0
2 0.01
2 0.01
1 0.17
2 0.01
2 0.01
2 0.01
7 0.0
2 0.01
7 0.0
1 0.19
1 0.17
1 0.19
1 0.19
1 0.17
1 0.18
1 0.18
1 0.19
4 0.03
1 0.19
4 0.02
4 0.02
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.01
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
4 0.02
4 0.02
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.03
4 0.02
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
5 0.0
5 0.01
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
4 0.01
12 0.01
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.0
16 0.01
16 0.01
16 0.0
16 0.0
16 0.0
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
16 0.01
4 0.04
4 0.04
4 0.03
4 0.07
11

KeyboardInterrupt: 

In [33]:
capture.release()
cv2.destroyAllWindows()
